In [1]:
import re
from copy import deepcopy
from os import path as osp
import open3d as o3d
from open3d.web_visualizer import draw
import sys
import os
import quaternion
import json
import quaternion
import math
import cv2

import numpy as np
import torch


lidar_path ='/home/server-003/workspace/data/superb_mobis/jupyter_apps/asset/0aafd40e-85d8-496b-bfb8-03afbe333caf/pointclouds_00000001.bin'
label_path ='/home/server-003/workspace/data/superb_mobis/jupyter_apps/label/0aafd40e-85d8-496b-bfb8-03afbe333caf/label_info.json'
vis_frame_num = 0
draw_box=[]

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [7]:

def convert_label_suite2kitti(label_path, vis_frame_num):
    file = open(label_path)
    jsonString = json.load(file)   
    frame_num = jsonString.get("label").get('asset').get("info").get("frame_count")
   
    try:
        jsonArray = jsonString.get("label_info").get('result').get("objects")
    except:
        print("can't find objects")
        return 
        
    for iter_num, list in enumerate(jsonArray):
        frames = list.get("frames")
        for frame in frames:
            idx = frame.get('num')
            if idx==vis_frame_num:
        
                ann = frame.get('annotation').get('coord')
                dim=[]
                height = np.round_(ann.get('size').get('z'),5)
                width = np.round_(ann.get('size').get('y'),5)
                length = np.round_(ann.get('size').get('x'),5)
                
                
                dim.append(width)
                dim.append(length)
                dim.append(height)
        
                lidar_coor_pose=[]
            
                lidar_coor_pose.append(np.round_(ann.get('position').get('x'),5))
                lidar_coor_pose.append(np.round_(ann.get('position').get('y'),5))
                lidar_coor_pose.append(np.round_(ann.get('position').get('z'),5)) #height/2 : gravity center
            

                q = np.quaternion(ann.get('rotation_quaternion').get('w'),ann.get('rotation_quaternion').get('x'),ann.get('rotation_quaternion').get('y'),ann.get('rotation_quaternion').get('z'))
                lidar_coor_RPY = np.round_(quaternion.as_euler_angles(q),5)
             
                lidar_coor_RPY[0]=0
                lidar_coor_RPY[2]=0
                rot_mat = o3d.geometry.get_rotation_matrix_from_xyz(lidar_coor_RPY)
            #     print(rot_mat)
                box3d = o3d.geometry.OrientedBoundingBox(lidar_coor_pose, rot_mat, dim)
                draw_box.append(box3d)     
    print(f"num of bbox : {len(draw_box)}")
    return


In [3]:
def load_bin(bin_path):
    points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)
    return points[:, :3]

In [4]:
convert_label_suite2kitti(label_path,vis_frame_num)
velo_points = load_bin(lidar_path)
cloud = o3d.geometry.PointCloud()
cloud.points = o3d.utility.Vector3dVector(velo_points[:,:3])
draw_box.append(cloud)

11


In [5]:
draw(draw_box)

[Open3D INFO] Window window_0 created.
[Open3D INFO] EGL headless mode enabled.


../src/intel/isl/isl.c:2105: FINISHME: ../src/intel/isl/isl.c:isl_surf_supports_ccs: CCS for 3D textures is disabled, but a workaround is available.


WebVisualizer(window_uid='window_0')

[Open3D INFO] ICE servers: {"stun:stun.l.google.com:19302", "turn:user:password@34.69.27.100:3478", "turn:user:password@34.69.27.100:3478?transport=tcp"}
FEngine (64 bits) created at 0x7f16eb572010 (threading is enabled)
[Open3D INFO] Set WEBRTC_STUN_SERVER environment variable add a customized WebRTC STUN server.
[Open3D INFO] WebRTC Jupyter handshake mode enabled.
EGL(1.5)
OpenGL(4.6)
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in PaintUniformColor, clipping to [0, 1]
[Open3D WARNING] invalid color in 

[000:000][97460] (stun_port.cc:96): Binding request timed out from 192.168.219.x:58517 (wlp147s0)
